In [8]:
%pip install -q opencv-python pillow imageio


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\aditya\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
import cv2
import os

video_path = "../sample_data/stempade_1.mp4"
output_dir = "extracted_frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

print(f"📼 FPS: {fps}, Total Frames: {frame_count}, Duration: {duration:.2f} seconds")

# Extract 1 frame per second
current_second = 0
frame_id = 0

while True:
    # Go to the frame at the given second
    frame_number = int(current_second * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    if not ret:
        break

    output_path = os.path.join(output_dir, f"frame_{current_second:02d}.jpg")
    cv2.imwrite(output_path, frame)
    print(f"✅ Saved frame at {current_second}s -> {output_path}")

    current_second += 1
    if current_second > duration:
        break

cap.release()


📼 FPS: 23.976, Total Frames: 97, Duration: 4.05 seconds
✅ Saved frame at 0s -> extracted_frames\frame_00.jpg
✅ Saved frame at 1s -> extracted_frames\frame_01.jpg
✅ Saved frame at 2s -> extracted_frames\frame_02.jpg
✅ Saved frame at 3s -> extracted_frames\frame_03.jpg
✅ Saved frame at 4s -> extracted_frames\frame_04.jpg


In [10]:
import os
import uuid
import json
import shutil
import requests
import smtplib
import imageio
from PIL import Image, ImageDraw
from email.message import EmailMessage
from dotenv import load_dotenv

load_dotenv()

# ------------------ CONFIG ------------------
API_URL            = "https://nsl6up9ztcem6h-8000.proxy.runpod.net/ask_image"
FRAME_DIR          = "extracted_frames"
ALERT_DIR          = "alerts"
PROMPT             = (
    "Analyze this image carefully. Do you see any signs of a stampede or crowd panic — "
    "such as people tightly packed, pushing, running, falling, or any chaos in the scene? "
    "Reply 'Yes' if any such signs are visible. Otherwise, reply 'No' only."
)
EMAIL_SENDER       = "ab0358031@gmail.com"
EMAIL_PASSWORD     = os.getenv("GOOGLE_APP_PASSWORD")
EMAIL_RECEIVER     = "ab0358031@gmail.com"
# --------------------------------------------

def get_verdict(score_percent: float) -> str:
    """Map risk percentage to verdict."""
    if score_percent <= 10:
        return "SAFE"
    elif score_percent <= 30:
        return "WATCH"
    else:
        return "ALERT"

def save_flagged_frames(event_id, flagged_frames):
    """Copy flagged frames to an alerts directory for record-keeping."""
    output_path = os.path.join(ALERT_DIR, f"event_{event_id}")
    os.makedirs(output_path, exist_ok=True)
    for frame in flagged_frames:
        src = os.path.join(FRAME_DIR, frame["frame"])
        dst = os.path.join(output_path, frame["frame"])
        shutil.copyfile(src, dst)
    return output_path

def generate_heatmap_gif(event_dir, flagged_frames, total_frames):
    """Generate a GIF overlaying a warning banner on flagged frames."""
    print("🎞️ Generating heat-map GIF...")
    flagged_set = {f["frame"] for f in flagged_frames}
    gif_frames = []
    for i in range(total_frames):
        fname = f"frame_{i:02d}.jpg"
        path = os.path.join(event_dir, fname)
        if not os.path.exists(path):
            continue
        img = Image.open(path).convert("RGB")
        if fname in flagged_set:
            draw = ImageDraw.Draw(img)
            draw.rectangle([(0, 0), (img.width, 40)], fill=(255, 0, 0))
            draw.text((10, 10), "⚠️ Stampede Risk", fill="white")
        gif_frames.append(img)
    gif_path = os.path.join(event_dir, "summary.gif")
    imageio.mimsave(gif_path, gif_frames, fps=1)
    print(f"✅ Saved GIF: {gif_path}")

def send_email(event_id, result, frame_dir):
    """Send an email alert with the risk summary and all flagged frames attached."""
    msg = EmailMessage()
    msg["Subject"] = f"🚨 ALERT: Crowd Panic Detected (Event {event_id})"
    msg["From"]    = EMAIL_SENDER
    msg["To"]      = EMAIL_RECEIVER

    # Build email body
    lines = [
        f"Event ID: {event_id}",
        f"Verdict: {result['verdict']}",
        f"Risk Score: {result['risk_score_percent']}%",
        "Flagged Frames:",
        "Location: 100, 100"
    ]
    for f in result["flagged_frames"]:
        lines.append(f" - {f['timestamp']} → {f['frame']}")
    lines.append("\nPlease investigate immediately.")
    msg.set_content("\n".join(lines))

    # Attach flagged images
    for f in result["flagged_frames"]:
        img_path = os.path.join(frame_dir, f["frame"])
        with open(img_path, "rb") as img:
            msg.add_attachment(img.read(),
                               maintype="image",
                               subtype="jpeg",
                               filename=f["frame"])
    # Attach GIF
    gif_path = os.path.join(frame_dir, "summary.gif")
    if os.path.exists(gif_path):
        with open(gif_path, "rb") as gif:
            msg.add_attachment(gif.read(),
                               maintype="image",
                               subtype="gif",
                               filename="summary.gif")

    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
            smtp.login(EMAIL_SENDER, EMAIL_PASSWORD)
            smtp.send_message(msg)
        print("📧 Email sent successfully!")
    except Exception as e:
        print(f"⚠️ Failed to send email: {e}")

def analyze_frames():
    total_frames   = 0
    yes_count      = 0
    flagged_frames = []
    event_id       = str(uuid.uuid4())[:8]

    # Analyze each frame
    for i, filename in enumerate(sorted(os.listdir(FRAME_DIR))):
        if not filename.lower().endswith(".jpg"):
            continue
        total_frames += 1
        ts = f"00:{i:02d}"
        path = os.path.join(FRAME_DIR, filename)
        with open(path, "rb") as f:
            files = {"image": (filename, f, "image/jpeg")}
            data  = {"prompt": PROMPT}
            print(f"📤 Sending {filename} (timestamp {ts})...")
            try:
                resp = requests.post(API_URL, files=files, data=data)
                resp.raise_for_status()
                text = resp.json().get("text", "").strip().lower()
                print(f"🧠 Response: {text}")
                if text == "yes":
                    yes_count += 1
                    flagged_frames.append({
                        "frame": filename,
                        "timestamp": ts,
                        "response": "Yes"
                    })
            except Exception as e:
                print(f"⚠️ Error on {filename}: {e}")

    # Compute risk
    score_percent = round((yes_count / total_frames) * 100, 2) if total_frames else 0.0
    verdict       = get_verdict(score_percent)

    result = {
        "event_id":          event_id,
        "verdict":           verdict,
        "risk_score_percent": score_percent,
        "yes_count":         yes_count,
        "total_frames":      total_frames,
        "flagged_frames":    flagged_frames
    }

    # Save JSON audit log
    frame_dir = save_flagged_frames(event_id, flagged_frames)
    with open(os.path.join(frame_dir, "result.json"), "w") as jf:
        json.dump(result, jf, indent=2)

    # Generate heat-map GIF
    generate_heatmap_gif(frame_dir, flagged_frames, total_frames)

    # Send email if ALERT
    if verdict == "ALERT":
        send_email(event_id, result, frame_dir)

    # Print summary to console
    print("\n📊 Final Risk Analysis:")
    print(json.dumps(result, indent=2))

if __name__ == "__main__":
    analyze_frames()


📤 Sending frame_00.jpg (timestamp 00:00)...
🧠 Response: yes
📤 Sending frame_01.jpg (timestamp 00:01)...
🧠 Response: yes
📤 Sending frame_02.jpg (timestamp 00:02)...
🧠 Response: yes
📤 Sending frame_03.jpg (timestamp 00:03)...
🧠 Response: yes
📤 Sending frame_04.jpg (timestamp 00:04)...
🧠 Response: yes.
🎞️ Generating heat-map GIF...
✅ Saved GIF: alerts\event_08677dcf\summary.gif
📧 Email sent successfully!

📊 Final Risk Analysis:
{
  "event_id": "08677dcf",
  "verdict": "ALERT",
  "risk_score_percent": 80.0,
  "yes_count": 4,
  "total_frames": 5,
  "flagged_frames": [
    {
      "frame": "frame_00.jpg",
      "timestamp": "00:00",
      "response": "Yes"
    },
    {
      "frame": "frame_01.jpg",
      "timestamp": "00:01",
      "response": "Yes"
    },
    {
      "frame": "frame_02.jpg",
      "timestamp": "00:02",
      "response": "Yes"
    },
    {
      "frame": "frame_03.jpg",
      "timestamp": "00:03",
      "response": "Yes"
    }
  ]
}
